https://upload.wikimedia.org/wikipedia/commons/c/ce/Logo-ITM-01.png

# Primero instalamos las librerías necesarias

In [1]:
!pip install dash jupyter-dash pandas plotly openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [2]:
!find drive -name "ingresos_gastos.xlsx"

drive/MyDrive/Colab Notebooks/ingresos_gastos.xlsx


In [3]:
!pip install pyngrok

# Creación del codigo que analiza la información contable compartida

In [4]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, dash_table
import threading
from pyngrok import ngrok
import numpy as np
from sklearn.linear_model import LinearRegression

# Configurar el token de autenticación de ngrok
ngrok.set_auth_token("2l5emrASGzaFh9GJUduLuJGWK2y_2XrXnm1GTy8RgRLQ2rd5J")

# Cargar los datos
df = pd.read_excel('drive/MyDrive/Colab Notebooks/ingresos_gastos.xlsx')

# Limpieza de los datos del Excel
df['Ingresos'] = pd.to_numeric(df['Ingresos'], errors='coerce')
df['Gastos'] = pd.to_numeric(df['Gastos'], errors='coerce')
df = df.dropna(subset=['Ingresos', 'Gastos'], how='all') # Eliminar los datos que puedan estar vacíos en ambas columnas, lo que indica que no lo debo tener en cuenta

# Inicializar la aplicación
app = Dash(__name__)

# Diseño del dashboard
app.layout = html.Div([
    html.H1('Dashboard de Ingresos y Gastos'),

    html.Div([
        dcc.Dropdown(
            id='mes-dropdown',
            options=[{'label': mes, 'value': mes} for mes in df['Mes'].unique()],
            multi=True,
            placeholder='Seleccione mes(es)',
            style={'width': '50%'}
        ),
        dcc.Dropdown(
            id='categoria-dropdown',
            options=[{'label': cat, 'value': cat} for cat in df['Categoría'].unique()],
            multi=True,
            placeholder='Seleccione categoría(s)',
            style={'width': '50%'}
        )
    ]),

    html.Div([
        dcc.Graph(id='ingresos-gastos-graph'),
        dcc.Graph(id='categoria-pie-chart'),
        dcc.Graph(id='margen-neto-graph'),
        html.Div(id='scorecard')
    ]),

    dash_table.DataTable(
        id='tabla-resumen',
        columns=[{"name": i, "id": i} for i in df.columns],
        style_table={'height': '300px', 'overflowY': 'auto'}
    )
])

# Callbacks para actualizar los gráficos y la tabla
@app.callback(
    [Output('ingresos-gastos-graph', 'figure'),
     Output('categoria-pie-chart', 'figure'),
     Output('margen-neto-graph', 'figure'),
     Output('scorecard', 'children'),
     Output('tabla-resumen', 'data')],
    [Input('mes-dropdown', 'value'),
     Input('categoria-dropdown', 'value')]
)
def update_graphs(meses_seleccionados, categorias_seleccionadas):
    dff = df.copy()

    if meses_seleccionados:
        dff = dff[dff['Mes'].isin(meses_seleccionados)]
    if categorias_seleccionadas:
        dff = dff[dff['Categoría'].isin(categorias_seleccionadas)]

    # Gráfico de barras apiladas de ingresos y gastos
    dff_melted = dff.melt(id_vars=['Mes', 'Categoría'], value_vars=['Ingresos', 'Gastos'],
                          var_name='Tipo', value_name='Monto')

    fig_bar = px.bar(dff_melted, x='Mes', y='Monto', color='Categoría', barmode='stack',
                     title='Ingresos y Gastos por Mes',
                     labels={'Monto': 'Monto', 'Mes': 'Mes', 'Categoría': 'Categoría', 'Tipo': 'Tipo'},
                     hover_data={'Categoría': True, 'Monto': True, 'Tipo': True})

    # Gráfico de torta por categoría
    dff_pie = df[df['Ingresos'] > 0]
    fig_pie = px.pie(dff_pie, values='Ingresos', names='Categoría', title='Distribución de Ingresos por Categoría')

    # Datos para la tabla
    tabla_data = dff.to_dict('records')

    # Cálculo del margen de ganancia neta
    dff_grouped = dff.groupby('Mes').agg({'Ingresos': 'sum', 'Gastos': 'sum'}).reset_index()
    dff_grouped['Margen de Ganancia Neta'] = (dff_grouped['Ingresos'] - dff_grouped['Gastos']) / dff_grouped['Ingresos']
    dff_grouped['Rentabilidad'] = dff_grouped['Margen de Ganancia Neta'] * 100

    # Gráfico de líneas de margen neto con línea de tendencia
    fig_line = px.line(dff_grouped, x='Mes', y='Margen de Ganancia Neta', markers=True, title='Margen de Ganancia Neta por Mes')

    # Regresión lineal para la línea de tendencia
    X = np.arange(len(dff_grouped)).reshape(-1, 1)  # Índices como variable independiente
    y = dff_grouped['Margen de Ganancia Neta'].values  # Margen de ganancia neta como variable dependiente
    model = LinearRegression().fit(X, y)
    trend_line = model.predict(X)

    fig_line.add_scatter(x=dff_grouped['Mes'], y=trend_line, mode='lines', line=dict(dash='dash'), name='Línea de Tendencia')

    # Scorecard
    margen_total = dff_grouped['Margen de Ganancia Neta'].mean()
    rentabilidad_status = 'Rentable' if margen_total > 0 else 'No Rentable'
    scorecard_text = f'Margen de Ganancia Neta Total: {margen_total:.2%} ({rentabilidad_status})'

    return fig_bar, fig_pie, fig_line, scorecard_text, tabla_data

# Función para ejecutar el servidor en un hilo separado
def run_server():
    app.run_server(port=8050, debug=False)

# Iniciar el servidor en un hilo separado
thread = threading.Thread(target=run_server)
thread.start()

# Configurar el túnel ngrok
public_url = ngrok.connect(8050)
print(f"Dash app URL: {public_url}")

<IPython.core.display.Javascript object>

Dash app URL: NgrokTunnel: "https://e8d0-35-201-205-61.ngrok-free.app" -> "http://localhost:8050"
